In [79]:
# main_solver.py

import gurobipy as gp
from gurobipy import GRB
import sys

# A classe DataParser permanece exatamente a mesma da versão anterior.
class DataParser:
    """
    Responsabilidade: Ler e analisar o arquivo de instância.
    (Código idêntico à resposta anterior)
    """
    def __init__(self, filepath):
        self.filepath = filepath
        self.orders = {}
        self.aisles = {}
        self.all_items = set()
        self.num_orders = 0
        self.num_items = 0
        self.num_aisles = 0
        self.lb = 0
        self.ub = 0

    def parse(self):
        print(f"INFO: Iniciando a análise do arquivo de instância: {self.filepath}")
        with open(self.filepath, 'r') as f:
            self.num_orders, self.num_items, self.num_aisles = map(int, f.readline().split())
            for i in range(self.num_orders):
                line = list(map(int, f.readline().split()))
                k = line[0]
                order_items = {}
                total_units = 0
                for j in range(k):
                    item_id = line[1 + 2*j]
                    quantity = line[2 + 2*j]
                    order_items[item_id] = quantity
                    total_units += quantity
                    self.all_items.add(item_id)
                self.orders[i] = {'items': order_items, 'total_units': total_units}
            for i in range(self.num_aisles):
                line = list(map(int, f.readline().split()))
                l = line[0]
                aisle_stock = {}
                for j in range(l):
                    item_id = line[1 + 2*j]
                    quantity = line[2 + 2*j]
                    aisle_stock[item_id] = quantity
                    self.all_items.add(item_id)
                self.aisles[i] = {'items': aisle_stock}
            self.lb, self.ub = map(int, f.readline().split())
        print("INFO: Análise do arquivo concluída com sucesso.")


class WaveOptimizer:
    """
    Versão atualizada da classe de otimização para incluir a lógica de parada por estagnação.
    """
    def __init__(self, data, lambda_penalty=1.0, stagnation_node_limit=5000):
        self.data = data
        self.model = gp.Model("Optimal_Wave_Selection")
        self.lambda_penalty = lambda_penalty
        self.solution = {}
        self.objective_value = None
        
        # --- LÓGICA DE ESTAGNAÇÃO ATUALIZADA ---
        # Design: O limite agora é baseado em NÓS, não em soluções encontradas.
        # Este valor precisa ser muito maior que o anterior.
        self.stagnation_node_limit = stagnation_node_limit
        self.stagnation_node_counter = 0
        self.last_objective = -float('inf')

    def _stagnation_callback(self, model, where):
        """
        Lógica Atualizada: O callback agora é acionado a cada nó explorado pelo Gurobi.
        Isso nos dá um feedback constante sobre o progresso da otimização.
        """
        # --- GATILHO MODIFICADO ---
        # Trocamos MIPSOL por MIPNODE. Agora, este código roda milhares de vezes durante
        # a otimização, nos dando um controle muito mais granular.
        if where == GRB.Callback.MIPNODE:
            # Verificamos se já existe uma solução incumbente. O MIPNODE_OBJBST só é válido
            # se o solver já encontrou pelo menos uma solução inteira (SolCount > 0).
            if model.cbGet(GRB.Callback.MIPNODE_SOLCNT) > 0:
                # Pega o valor do MELHOR objetivo encontrado até agora na árvore.
                current_best_obj = model.cbGet(GRB.Callback.MIPNODE_OBJBST)

                # A lógica de comparação permanece a mesma.
                if current_best_obj > self.last_objective + 1e-3:
                    # Se o melhor objetivo global melhorou, zeramos o contador e guardamos o novo valor.
                    self.last_objective = current_best_obj
                    self.stagnation_node_counter = 0
                else:
                    # Se o melhor objetivo global não melhorou, incrementamos o contador de nós estagnados.
                    self.stagnation_node_counter += 1
                
                # A verificação de parada agora usa o novo contador baseado em nós.
                if self.stagnation_node_counter >= self.stagnation_node_limit:
                    print(f"\nINFO: Limite de estagnação ({self.stagnation_node_limit} nós sem melhoria) atingido. Terminando a otimização.")
                    model.terminate()

    def build_and_solve(self):
        """
        Método principal atualizado para passar a função de callback para o Gurobi.
        """
        print("INFO: Construindo o modelo de otimização...")
        self._create_decision_variables()
        print('\n Terminamos de criar variáveis \n')
        self._set_objective_function()
        print('\n Terminamos de criar objective function \n')
        self._add_constraints()
        print('\n Terminamos de criar constraints \n')

        print("INFO: Otimização iniciada. Buscando a melhor solução...")
        
        # A chamada é a mesma, mas o comportamento do callback interno mudou.
        self.model.optimize(self._stagnation_callback)
        
        self._extract_solution()

    def _create_decision_variables(self):
        self.select_order = self.model.addVars(
            self.data.orders.keys(), vtype=GRB.BINARY, name="SelectOrder"
        )
        self.visit_aisle = self.model.addVars(
            self.data.aisles.keys(), vtype=GRB.BINARY, name="VisitAisle"
        )

    def _set_objective_function(self):
        total_items_in_wave = gp.quicksum(
            self.data.orders[o]['total_units'] * self.select_order[o] for o in self.data.orders
        )
        total_aisles_visited = gp.quicksum(self.visit_aisle)
        self.model.setObjective(
            total_items_in_wave - self.lambda_penalty * total_aisles_visited, GRB.MAXIMIZE
        )

    def _add_constraints(self):
        total_items_in_wave = gp.quicksum(
            self.data.orders[o]['total_units'] * self.select_order[o] for o in self.data.orders
        )
        self.model.addConstr(total_items_in_wave >= self.data.lb, "Min_Wave_Size")
        self.model.addConstr(total_items_in_wave <= self.data.ub, "Max_Wave_Size")

        for item_id in self.data.all_items:
            total_demand = gp.quicksum(
                self.data.orders[o]['items'].get(item_id, 0) * self.select_order[o] for o in self.data.orders
            )
            total_stock = gp.quicksum(
                self.data.aisles[a]['items'].get(item_id, 0) * self.visit_aisle[a] for a in self.data.aisles
            )
            self.model.addConstr(total_demand <= total_stock, f"Stock_Constraint_{item_id}")


    def _extract_solution(self):
        if self.model.SolCount > 0:
            print(f"INFO: Solução final encontrada com valor objetivo = {self.model.ObjVal:.2f}")
            selected_orders = [o for o, var in self.select_order.items() if var.X > 0.5]
            visited_aisles = [a for a, var in self.visit_aisle.items() if var.X > 0.5]
            
            total_items = sum(self.data.orders[o]['total_units'] for o in selected_orders)
            num_aisles = len(visited_aisles)
            original_obj_val = (total_items / num_aisles) if num_aisles > 0 else 0
            
            self.solution = {
                'selected_orders': selected_orders,
                'visited_aisles': visited_aisles,
            }
            self.objective_value = self.model.ObjVal
            
            print(f"INFO: Pedidos selecionados ({len(selected_orders)}): {selected_orders[:10]}...")
            print(f"INFO: Corredores visitados ({len(visited_aisles)}): {visited_aisles[:10]}...")
            print(f"INFO: Total de itens na wave: {total_items}")
            print(f"INFO: Valor objetivo original (Itens/Corredor): {original_obj_val:.4f}")

        else:
            print("ERRO: Nenhuma solução viável foi encontrada.")
            self.solution = None

In [80]:
# A função main permanece a mesma, ela apenas orquestra o processo.
def main(instance_file, output_file):
    """
    Função principal que orquestra todo o processo.
    """
    parser = DataParser(instance_file)
    parser.parse()
    
    # Podemos passar o limite de estagnação e o lambda como parâmetros
    lambda_param = 2.0
    stagnation_nodes = 200
    
    optimizer = WaveOptimizer(
        parser, 
        lambda_penalty=lambda_param, 
        stagnation_node_limit=stagnation_nodes
    )
    optimizer.build_and_solve()

    if optimizer.solution:
        print(f"INFO: Escrevendo a solução para o arquivo: {output_file}")
        with open(output_file, 'w') as f:
            selected_orders = optimizer.solution['selected_orders']
            visited_aisles = optimizer.solution['visited_aisles']
            
            f.write(f"{len(selected_orders)}\n")
            for order in sorted(selected_orders):
                f.write(f"{order}\n")
            
            f.write(f"{len(visited_aisles)}\n")
            for aisle in sorted(visited_aisles):
                f.write(f"{aisle}\n")
        
        print("INFO: Processo concluído com sucesso.")
    else:
        print("ERRO: O processo terminou sem uma solução para escrever.")

In [81]:
import os
path_to_instances_b = './datasets/b'
todas_instancias = os.listdir(path_to_instances_b)

In [82]:
from checker_biblioteca.checker import run_checker
'''
for i in range(len(todas_instancias)):
    instance_file = f'{path_to_instances_b}/{todas_instancias[i]}'
    output_file = f'./{todas_instancias[i]}'
    #if __name__ == "__main__":
    main(instance_file, output_file)

    run_checker(instance_file, output_file)
'''

'\nfor i in range(len(todas_instancias)):\n    instance_file = f\'{path_to_instances_b}/{todas_instancias[i]}\'\n    output_file = f\'./{todas_instancias[i]}\'\n    #if __name__ == "__main__":\n    main(instance_file, output_file)\n\n    run_checker(instance_file, output_file)\n'

In [83]:
instance_file = './datasets/b/instance_0013.txt'
output_file = 'solution_0013.txt'
main(instance_file, output_file)
run_checker(instance_file, output_file)

INFO: Iniciando a análise do arquivo de instância: ./datasets/b/instance_0013.txt
INFO: Análise do arquivo concluída com sucesso.
INFO: Construindo o modelo de otimização...

 Terminamos de criar variáveis 


 Terminamos de criar objective function 



C:\Users\Danone\AppData\Local\Temp\ipykernel_3764\474722181.py:132: DeprecationWarning: Calling quicksum on a tupledict is deprecated, use .sum() instead.
  total_aisles_visited = gp.quicksum(self.visit_aisle)


KeyboardInterrupt: 